In [5]:
from azureml.core import Workspace, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id='660b3398-b80e-49d2-bc5b-ac1dc93b5254')

ws = Workspace.get(
      name='quick-starts-ws-126416',
      subscription_id='ebee3a56-4c54-406a-b732-174015826780',
      resource_group='aml-quickstarts-126416'
)
# ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
Workspace name: quick-starts-ws-126416
Azure region: southcentralus
Subscription id: ebee3a56-4c54-406a-b732-174015826780
Resource group: aml-quickstarts-126416


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_name = "train-cluster"
vm_size = "Standard_D2_V2"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target:', compute_name)
else:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=0, max_nodes=4)

    # create the compute target
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())

Found compute target: train-cluster


In [20]:
import os

from azureml.train.sklearn import SKLearn
#from azureml.train.estimator import Estimator
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform

# Specify parameter sampler
ps = RandomParameterSampling({
    'C': uniform(0.01, 100),
    'max_iter': choice(range(10, 500))
})
### YOUR CODE HERE ###

# Specify a Policy: 
# the early termination policy is applied at every interval when metrics are reported, starting at evaluation interval 5. 
# Any run whose best metric is less than (1/(1+0.1) or 91% of the best performing run will be terminated 
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)  ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = Estimator(
#    source_directory=os.path.join('.', 'script'),
#    entry_script='train.py',
#    compute_target=compute_target
#)
estimator = SKLearn(
    source_directory=os.path.join('.', 'script'), 
    entry_script='train.py',
    #script_params='',
    compute_target=compute_target,
    #pip_packages=['joblib']
)
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=4)
### YOUR CODE HERE ###

In [21]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(hyperdrive_config)
### YOUR CODE HERE ###

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [22]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [23]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_5d5dda37-df0c-4ef0-ad1b-6f9061987852,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [24]:
run.wait_for_completion(show_output=True)

RunId: HD_5d5dda37-df0c-4ef0-ad1b-6f9061987852
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_5d5dda37-df0c-4ef0-ad1b-6f9061987852?wsid=/subscriptions/ebee3a56-4c54-406a-b732-174015826780/resourcegroups/aml-quickstarts-126416/workspaces/quick-starts-ws-126416

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-16T07:38:15.822021][API][INFO]Experiment created<END>\n""<START>[2020-11-16T07:38:16.683959][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-16T07:38:16.995345][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-16T07:38:17.8299686Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_5d5dda37-df0c-4ef0-ad1b-6f9061987852
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_5d5dda37-df0c-4ef0-ad1b-6f9061987852?wsid=/subscriptions/ebee3

{'runId': 'HD_5d5dda37-df0c-4ef0-ad1b-6f9061987852',
 'target': 'train-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2020-11-16T07:38:15.618943Z',
 'endTimeUtc': '2020-11-16T07:47:10.526054Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": {\n      "type": null,\n      "message": "[(https://southcentralus.experiments.azureml.net/subscriptions/ebee3a56-4c54-406a-b732-174015826780/resourceGroups/aml-quickstarts-126416/providers/Microsoft.Mach

In [25]:
# Get your best run and save the model from that run.
best_run = run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:', parameter_values[3])
print('\n keep probability:', parameter_values[5])
print('\n batch size:', parameter_values[7])
### YOUR CODE HERE ###

Exception: No best run was found

In [26]:
import joblib
# Save the model from that run.
print(best_run.get_file_names())
#joblib.dump(svm_model_linear, 'hyperdrive_model.joblib')
### YOUR CODE HERE ###

AttributeError: 'NoneType' object has no attribute 'get_file_names'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###